<a href="https://colab.research.google.com/github/pablosoria7/Python/blob/main/DeptosBariloche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import sys
import re

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

In [ ]:
url = 'https://inmuebles.mercadolibre.com.ar/departamentos/venta/rio-negro/bariloche/'

In [ ]:

def get_links(url_base):

    links = []
    driver.get(url_base)

    # Comenzar desde el numero de página
    get_page_num = driver.find_element(By.CSS_SELECTOR, "a.andes-pagination__link.shops__pagination-link.ui-search-link")
    page_number = int(re.search("_Desde_(\d+)_", get_page_num.get_attribute('href')).group(1))
    while True:
        current_url = f"{url_base}/_Desde_{page_number}_NoIndex_True"
        driver.get(current_url)

        # Encontrar los enlaces en la página actual
        url_elements = driver.find_elements(By.CSS_SELECTOR, "a.andes-pagination__link.shops__pagination-link.ui-search-link")
        current_links = [x.get_attribute('href') for x in url_elements]

        # Si no hay más enlaces en la página actual, stop
        if not current_links:
            break

        # Agregar los enlaces de la página actual a la lista general de enlaces
        links.extend(current_links)

        # Incrementar el número de página para la siguiente iteración
        page_number += 49

    return list(set(links))# elementos únicos


# Obtener todos los enlaces
all_links = get_links(url)
sorted_urls = sorted(all_links, key=lambda url: int(url.split('_Desde_')[1].split('_')[0]) if '_Desde_' in url else 0)


# Departamentos

In [ ]:
def get_products(link):
    driver.get(link)
    sleep(1)
    #Elementos
    m2_amb = driver.find_elements(By.CSS_SELECTOR, "ul.ui-search-card-attributes.ui-search-item__group__element.shops__items-group-details")
    p = driver.find_elements(By.CSS_SELECTOR, "span.andes-money-amount__fraction")
    loc = driver.find_elements(By.CSS_SELECTOR, "span.ui-search-item__group__element.ui-search-item__location.shops__items-group-details")
    a = driver.find_elements(By.CSS_SELECTOR, "li.ui-search-card-attributes__attribute:nth-child(2)")
    s = driver.find_elements(By.CSS_SELECTOR, "span.andes-money-amount__currency-symbol")
    links = driver.find_elements(By.CSS_SELECTOR, "a.ui-search-item__group__element.shops__items-group-details.ui-search-link")
    item_title = driver.find_elements(By.CSS_SELECTOR, "h2.ui-search-item__title.shops__item-title")
    #Limpieza
    upd_m2_amb = [item.text + '\n ' if '\n' not in item.text else item.text for item in m2_amb]
    m2_amb = [i.split('\n') for i in upd_m2_amb]
    m2 = [x[0].split(' m² ') for x in m2_amb]
    href = [item.get_attribute('href') for item in links]
    quote = "tracking_id="
    #Variables
    dict_ = {
    'fecha': [pd.Timestamp.now()]*len(href),
    'item_title': [x.text for x in item_title],
    'm2' : [i[0] for i in m2],
    'm2_ct' : [i[1] for i in m2],
    'precio' : [item.text for item in p],
    'location' : [item.text for item in loc],
    'ambientes' : [x[1].replace(' amb.','').replace( ' ambs.','') for x in m2_amb],
    'urls' : href,
    'tracking_id' : [x[x.find(quote) + len(quote):]for x in href],
    'num_publicacion': [re.search(r'MLA-(\d+)-', num).group(1) for num in href],
    'unidad_monetaria' : [item.text for item in s]
                }

    sleep(1)
    return pd.DataFrame(dict_)


In [ ]:
df = pd.concat([get_products(x) for x in sorted_urls])
df

,fecha,item_title,m2,m2_ct,precio,location,ambientes,urls,tracking_id,num_publicacion,unidad_monetaria
0,2023-08-16 02:58:10.527613,Vista Al Lago Melipal Apto Turismo A 100m De B...,106,cubiertos,350.000,"Tronador 4650, Melipal, San Carlos De Bariloch...",3,https://departamento.mercadolibre.com.ar/MLA-1...,b7cda4dc-9590-4e5f-9493-888b171ac9e1,1470411034,U$S
1,2023-08-16 02:58:10.527613,Departamento En Venta - Bariloche - Id: 66320,83,cubiertos,59.000,"Anasagaasti 1900, Centro, Bariloche, Río Negro",4,https://departamento.mercadolibre.com.ar/MLA-1...,b7cda4dc-9590-4e5f-9493-888b171ac9e1,1471003364,U$S
2,2023-08-16 02:58:10.527613,Departamento En Venta - Bariloche- Vista Al La...,71,cubiertos,145.000,"Frey 22, Centro, Bariloche, Río Negro",3,https://departamento.mercadolibre.com.ar/MLA-1...,b7cda4dc-9590-4e5f-9493-888b171ac9e1,1471015430,U$S
3,2023-08-16 02:58:10.527613,Departamento En Venta En Pozo - Bariloche - Id...,65,cubiertos,145.000,"Morel 315, Peumayén, Bariloche, Río Negro",2 dorms.,https://departamento.mercadolibre.com.ar/MLA-1...,b7cda4dc-9590-4e5f-9493-888b171ac9e1,1470942398,U$S
4,2023-08-16 02:58:10.527613,Departamento En Venta En Pozo - Bariloche - Id...,65,cubiertos,130.000,"Morel 315, Peumayén, Bariloche, Río Negro",3,https://departamento.mercadolibre.com.ar/MLA-1...,b7cda4dc-9590-4e5f-9493-888b171ac9e1,1470978300,U$S
...,...,...,...,...,...,...,...,...,...,...,...
43,2023-08-16 02:58:20.383295,Departamento En Venta En B° El Cóndor.,44,cubiertos,65.000,"Maria Elena Walsh, Esquina Guido Spano., San C...",3,https://departamento.mercadolibre.com.ar/MLA-1...,a0317572-73a7-4cd7-adbc-7c80818f312a,1370626509,U$S
44,2023-08-16 02:58:20.383295,Departamento Apto Turístico. 1 Dormitorio Con ...,40,cubiertos,90.000,"Esquina Mitre Y Calle Los Pehuenes., San Carlo...",2,https://departamento.mercadolibre.com.ar/MLA-1...,a0317572-73a7-4cd7-adbc-7c80818f312a,1371634251,U$S
45,2023-08-16 02:58:20.383295,¡precio Reducido! Departamento En Venta Ubicad...,82,cubiertos,165.000,"Villegas 300, San Carlos De Bariloche, Bariloc...",3,https://departamento.mercadolibre.com.ar/MLA-1...,a0317572-73a7-4cd7-adbc-7c80818f312a,1376776331,U$S
46,2023-08-16 02:58:20.383295,"Venta Hermoso Departamento De 1 Dormitorio, En...",37,cubiertos,120.000,"20 De Junio Al 700, San Carlos De Bariloche, B...",2,https://departamento.mercadolibre.com.ar/MLA-1...,a0317572-73a7-4cd7-adbc-7c80818f312a,1468349170,U$S


# Tablas principales

In [ ]:

def get_tables(url_products):
    driver.get(url_products)
    df_tables = pd.read_html(driver.page_source)[0]
    df_tables.set_index(0, inplace=True)
    df_tables = df_tables.T
    df_tables['num_publicacion'] = re.search(r'MLA-(\d+)-', url_products).group(1)

    return df_tables

,Superficie total,Superficie cubierta,Superficie de balcón,Ambientes,Dormitorios,Baños,Cocheras,Depósitos,Cantidad de pisos,Departamentos por piso,Tipo de departamento,Disposición,Orientación,Antigüedad,Expensas,num_publicacion
1,135 m²,106 m²,20 m²,3,2,2,1,0,2,1,Dúplex,Frente,Norte,4 años,0 ARS,1470411034


In [ ]:
principales = pd.concat([get_tables(x) for x in df.urls])
principales

,Superficie total,Superficie cubierta,Superficie de balcón,Ambientes,Dormitorios,Baños,Cocheras,Depósitos,Cantidad de pisos,Departamentos por piso,Tipo de departamento,Disposición,Orientación,Antigüedad,Expensas,num_publicacion,Número de piso de la unidad,Fecha de entrega,Unidades totales
1,135 m²,106 m²,20 m²,3,2,2,1,0,2,1,Dúplex,Frente,Norte,4 años,0 ARS,1470411034,NaN,NaN,NaN
1,83 m²,83 m²,0 m²,4,3,1,0,0,0,0,NaN,NaN,Norte,20 años,0 ARS,1471003364,0,NaN,NaN
1,0 m²,71 m²,0 m²,3,2,2,0,0,0,0,NaN,NaN,Norte,50 años,0 ARS,1471015430,0,NaN,NaN
1,0 m²,65 m²,0 m²,0,2,1,1,0,0,0,NaN,NaN,Norte,0 años,0 ARS,1470942398,0,NaN,NaN
1,0 m²,65 m²,0 m²,3,2,1,1,0,0,0,NaN,NaN,Norte,0 años,0 ARS,1470978300,0,NaN,NaN
1,0 m²,30.05 m²,0 m²,1,0,1,0,0,0,0,NaN,NaN,Norte,0 años,0 ARS,1470754220,0,NaN,NaN
1,0 m²,54.09 m²,0 m²,2,1,1,0,0,0,0,NaN,NaN,Norte,0 años,0 ARS,1470903520,0,NaN,NaN
1,0 m²,54.09 m²,0 m²,2,1,1,0,0,0,0,NaN,NaN,Norte,0 años,0 ARS,1470693038,0,NaN,NaN
1,45 m²,43 m²,2 m²,2,1,1,0,NaN,1,3,Departamento,NaN,NaN,3 años,7800 ARS,1472017188,4,NaN,NaN
1,39.9 m²,39.9 m²,0 m²,1,0,1,0,0,3,0,NaN,NaN,Norte,22 años,1500 ARS,1371588061,0,NaN,NaN


# Big DataFrame

In [ ]:
merg_df = pd.merge(df, principales, on='num_publicacion')
merg_df.info()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
fecha,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,...,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513,2023-08-16 00:50:42.869513
item_title,Vista Al Lago Melipal Apto Turismo A 100m De B...,Departamento En Venta - Bariloche - Id: 66320,Departamento En Venta - Bariloche- Vista Al La...,Departamento En Venta En Pozo - Bariloche - Id...,Departamento En Venta En Pozo - Bariloche - Id...,Departamento En Venta - Cochera Cubierta - Bar...,Departamento En Venta - Cochera Cubierta - Bar...,Departamento En Venta - Bariloche - Id:66580,Venta De Excelente Departamento Dos Ambientes ...,Departamento En Venta - Bariloche - Apto Turis...,...,Departamento En Venta - Villa Catedral - Baril...,Departamento En Venta 2 Ambientes En Km 4.900 ...,Departamento En Venta De 2 Ambientes En Villa ...,Bustillo Av. Km 1 1º G,Capitalinas Bariloche,Departamento En Venta Frente Al Lago - Bariloche,Departamento En Venta Con Vista Al Lago - Fina...,Depto C/coch Cub. Amoblado Completo. Permuto P...,Departamento Amoblado En Venta - Centro - Bari...,Departamento En Venta De 2 Ambientes En Villa ...
m2,106,83,71,65,65,30,54,54,43,39,...,25,48,49,73,73 (- de 94),83,43,50,100,48
m2_ct,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,...,cubiertos,cubiertos,cubiertos,cubiertos,cubiertos,totales,cubiertos,cubiertos,cubiertos,cubiertos
precio,350.000,59.000,145.000,145.000,130.000,100.000,150.000,140.000,107.000,58.000,...,78.500,176.000,165.000,410.784,363.995,318.022,91.160,160.000,150.000,180.000
location,"Tronador 4650, Melipal, San Carlos De Bariloch...","Anasagaasti 1900, Centro, Bariloche, Río Negro","Frey 22, Centro, Bariloche, Río Negro","Morel 315, Peumayén, Bariloche, Río Negro","Morel 315, Peumayén, Bariloche, Río Negro","Campichuelo 748, Belgrano, Bariloche, Río Negro","Campichuelo 748, Belgrano, Bariloche, Río Negro","Campichuelo 748, Belgrano, Bariloche, Río Negro","Palacios 654, San Carlos De Bariloche, Río Neg...","Mendoza 264, San Carlos De Bariloche, Bariloch...",...,"Piedra Del Condor 512, San Carlos De Bariloche...","Av. Bustillo 4.900, San Carlos De Bariloche, B...","Los Notros 1530 R8402, Dina Huapi, Rio Negro, ...","Bustillo Av. Km 1, Bariloche, Río Negro","Bariloche, Río Negro","Capitalinas Av. Bustillo Km 1, Bariloche, Río ...","Tronador 1721, San Carlos De Bariloche, Barilo...","Av. De Los Pioneros 195 3° F, San Carlos De Ba...","Salta 476, Centro, Bariloche, Río Negro","Los Notros 1530 R8402, Dina Huapi, Rio Negro, ..."
ambientes,3,4,3,2 dorms.,3,1,2,2,2,1,...,,2,2,2,2 - 3,2,2,2,4,2
urls,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,https://departamento.mercadolibre.com.ar/MLA-1...,h

In [ ]:
merg_df.head(5)